In [4]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch import nn
import sklearn
import pandas as pd
import json
from tqdm.auto import tqdm
import re

/root/repos/pytorch-learn/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [154]:
all_chars = set()
data = []
with open("../data/text8/text8") as f:
    # read chunks but not by line since text8 doesn't have lines
    for i, chunk in tqdm(enumerate(iter(lambda: f.read(1024), ''))):
        # words = nltk.word_tokenize(chunk)
        chars = [c for c in chunk]
        all_chars.update(chars)
        for char in chars:
            data.append(char)
        if i > 50000:
            break
        

50001it [00:07, 6973.79it/s]


In [155]:
# write data to a file text8-small
with open("../data/text8/text8-small", "w") as f:
    f.write("".join(data))

In [106]:
len(all_chars)

27

In [107]:
data[:100]

[' ',
 'a',
 'n',
 'a',
 'r',
 'c',
 'h',
 'i',
 's',
 'm',
 ' ',
 'o',
 'r',
 'i',
 'g',
 'i',
 'n',
 'a',
 't',
 'e',
 'd',
 ' ',
 'a',
 's',
 ' ',
 'a',
 ' ',
 't',
 'e',
 'r',
 'm',
 ' ',
 'o',
 'f',
 ' ',
 'a',
 'b',
 'u',
 's',
 'e',
 ' ',
 'f',
 'i',
 'r',
 's',
 't',
 ' ',
 'u',
 's',
 'e',
 'd',
 ' ',
 'a',
 'g',
 'a',
 'i',
 'n',
 's',
 't',
 ' ',
 'e',
 'a',
 'r',
 'l',
 'y',
 ' ',
 'w',
 'o',
 'r',
 'k',
 'i',
 'n',
 'g',
 ' ',
 'c',
 'l',
 'a',
 's',
 's',
 ' ',
 'r',
 'a',
 'd',
 'i',
 'c',
 'a',
 'l',
 's',
 ' ',
 'i',
 'n',
 'c',
 'l',
 'u',
 'd',
 'i',
 'n',
 'g',
 ' ',
 't']

In [108]:
data

[' ',
 'a',
 'n',
 'a',
 'r',
 'c',
 'h',
 'i',
 's',
 'm',
 ' ',
 'o',
 'r',
 'i',
 'g',
 'i',
 'n',
 'a',
 't',
 'e',
 'd',
 ' ',
 'a',
 's',
 ' ',
 'a',
 ' ',
 't',
 'e',
 'r',
 'm',
 ' ',
 'o',
 'f',
 ' ',
 'a',
 'b',
 'u',
 's',
 'e',
 ' ',
 'f',
 'i',
 'r',
 's',
 't',
 ' ',
 'u',
 's',
 'e',
 'd',
 ' ',
 'a',
 'g',
 'a',
 'i',
 'n',
 's',
 't',
 ' ',
 'e',
 'a',
 'r',
 'l',
 'y',
 ' ',
 'w',
 'o',
 'r',
 'k',
 'i',
 'n',
 'g',
 ' ',
 'c',
 'l',
 'a',
 's',
 's',
 ' ',
 'r',
 'a',
 'd',
 'i',
 'c',
 'a',
 'l',
 's',
 ' ',
 'i',
 'n',
 'c',
 'l',
 'u',
 'd',
 'i',
 'n',
 'g',
 ' ',
 't',
 'h',
 'e',
 ' ',
 'd',
 'i',
 'g',
 'g',
 'e',
 'r',
 's',
 ' ',
 'o',
 'f',
 ' ',
 't',
 'h',
 'e',
 ' ',
 'e',
 'n',
 'g',
 'l',
 'i',
 's',
 'h',
 ' ',
 'r',
 'e',
 'v',
 'o',
 'l',
 'u',
 't',
 'i',
 'o',
 'n',
 ' ',
 'a',
 'n',
 'd',
 ' ',
 't',
 'h',
 'e',
 ' ',
 's',
 'a',
 'n',
 's',
 ' ',
 'c',
 'u',
 'l',
 'o',
 't',
 't',
 'e',
 's',
 ' ',
 'o',
 'f',
 ' ',
 't',
 'h',
 'e',
 ' ',
 'f'

In [109]:
len(all_chars)

27

In [110]:
str_to_token = {word: i for i, word in enumerate(all_chars)}
token_to_str = {i: word for i, word in enumerate(all_chars)}
tokenizer = lambda x: [str_to_token[word] for word in x]
detokenizer = lambda x: [token_to_str[word] for word in x]

first_few = data[:20]
print(f"first ten works {first_few}")
print(f"first ten tokens {tokenizer(first_few)}")
print(f"first ten detokens {detokenizer(tokenizer(first_few))}")


first ten works [' ', 'a', 'n', 'a', 'r', 'c', 'h', 'i', 's', 'm', ' ', 'o', 'r', 'i', 'g', 'i', 'n', 'a', 't', 'e']
first ten tokens [0, 13, 11, 13, 23, 17, 26, 16, 8, 9, 0, 7, 23, 16, 15, 16, 11, 13, 4, 22]
first ten detokens [' ', 'a', 'n', 'a', 'r', 'c', 'h', 'i', 's', 'm', ' ', 'o', 'r', 'i', 'g', 'i', 'n', 'a', 't', 'e']


In [111]:
CONTEXT_WINDOW = 100

In [112]:
# take chunks of exactly CONTEXT_WINDOW words and chuck everything else and flatten to a single list of these
examples = [tokenizer(data[i:i+CONTEXT_WINDOW]) for i in range(0, len(data), CONTEXT_WINDOW) if i+CONTEXT_WINDOW < len(data)]

In [113]:
examples[0]

[0,
 13,
 11,
 13,
 23,
 17,
 26,
 16,
 8,
 9,
 0,
 7,
 23,
 16,
 15,
 16,
 11,
 13,
 4,
 22,
 10,
 0,
 13,
 8,
 0,
 13,
 0,
 4,
 22,
 23,
 9,
 0,
 7,
 19,
 0,
 13,
 14,
 21,
 8,
 22,
 0,
 19,
 16,
 23,
 8,
 4,
 0,
 21,
 8,
 22,
 10,
 0,
 13,
 15,
 13,
 16,
 11,
 8,
 4,
 0,
 22,
 13,
 23,
 5,
 2,
 0,
 25,
 7,
 23,
 1,
 16,
 11,
 15,
 0,
 17,
 5,
 13,
 8,
 8,
 0,
 23,
 13,
 10,
 16,
 17,
 13,
 5,
 8,
 0,
 16,
 11,
 17,
 5,
 21,
 10,
 16,
 11,
 15,
 0,
 4]

In [114]:
torch.tensor(examples).shape

torch.Size([512020, 100])

In [115]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = torch.tensor(data, dtype=torch.long, device=device)

    def __len__(self):
        return len(self.data) * CONTEXT_WINDOW

    def __getitem__(self, index):
        example_idx = index // CONTEXT_WINDOW
        inside_example_idx = index % CONTEXT_WINDOW
        x = torch.zeros(CONTEXT_WINDOW, dtype=torch.long, device=device)
        x[CONTEXT_WINDOW-inside_example_idx:] = self.data[example_idx][:inside_example_idx]
        y = self.data[example_idx][inside_example_idx]
        return x, y

In [129]:
split = int(0.8 * len(examples))
# shuffle order first
# examples = sklearn.utils.shuffle(examples)
train_rows = examples[:split]
test_rows = examples[split:]
len(train_rows), len(test_rows)

(409616, 102404)

In [130]:
train_data = CustomDataset(train_rows)
test_data = CustomDataset(test_rows)

len(train_data), len(test_data)

(40961600, 10240400)

In [121]:
# sample a random train data point
i = np.random.randint(len(train_data))
x, y = train_data[i]
x, y

(tensor([ 0,  6, 22, 11,  0, 14, 16, 23,  4, 26,  8,  0,  7, 11, 22,  0, 12, 22,
         23,  7,  0, 12, 22, 23,  7,  0, 12, 22, 23,  7,  0, 24,  7, 24, 21,  5,
         13,  4, 16,  7, 11,  0,  4, 25,  7,  0, 12, 22, 23,  7,  0, 12, 22, 23,
          7,  0, 19, 16,  6, 22,  0, 22,  8,  4,  0, 10, 22, 13,  4, 26,  0, 23,
         13,  4, 22,  0,  7, 11, 22,  0, 19, 16,  6, 22,  0, 19,  7, 21, 23,  0,
         10, 22, 13,  4, 26,  8,  0,  7, 11, 22]),
 tensor(0))

In [119]:
from torch.utils.data import DataLoader
BATCH_SIZE = 10

train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

train_features_batch, train_labels_batch = next(iter(train_dataloader))
train_features_batch.shape, train_labels_batch.shape

(torch.Size([10, 100]), torch.Size([10]))

In [156]:
x = torch.randint(3, (1,3), dtype=torch.long).to(device)
print(f"x {x}")
embedding_dim = 3
embedding = nn.Embedding(len(all_chars), embedding_dim).to(device)
x = embedding(x).to(device)
print(f"after embedding {x}")

hidden_units = 4
keys = nn.Linear(embedding_dim, hidden_units).to(device)
queries = nn.Linear(embedding_dim, hidden_units).to(device)
values = nn.Linear(embedding_dim, embedding_dim).to(device)
K: torch.Tensor = keys(x)
Q: torch.Tensor = queries(x)
V: torch.Tensor = values(x)
print(f"keys shape {K}")
print(f"queries shape {Q}")
print(f"values shape {V}")

torch.matmul(Q, K.transpose(-2, -1))

x tensor([[2, 0, 1]])
after embedding tensor([[[ 2.2838,  0.3323, -0.0921],
         [-0.0763,  0.6336,  0.8634],
         [ 1.7532, -1.5495,  0.1196]]], grad_fn=<EmbeddingBackward0>)
keys shape tensor([[[-0.0933, -0.1292,  0.5140,  0.4969],
         [ 0.3470, -0.0704, -0.5814, -0.5986],
         [-0.4484,  0.6269,  0.5217,  0.0563]]], grad_fn=<ViewBackward0>)
queries shape tensor([[[-1.2417, -0.1688,  0.8942, -1.4008],
         [-1.0212,  0.1979,  0.6813, -0.2369],
         [-0.2959,  0.4723,  0.1512, -0.8121]]], grad_fn=<ViewBackward0>)
values shape tensor([[[ 1.1759, -0.4907, -0.5228],
         [ 0.1848, -0.0240,  0.0080],
         [ 1.2319,  0.1147,  0.0014]]], grad_fn=<ViewBackward0>)


tensor([[[-0.0988, -0.1003,  0.8386],
         [ 0.3022, -0.6225,  0.9241],
         [-0.3592,  0.2623,  0.4619]]], grad_fn=<UnsafeViewBackward0>)

In [123]:
class AttentionHead(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int) -> None:
        super().__init__()
        self.keys = nn.Linear(input_shape, hidden_units)
        self.queries = nn.Linear(input_shape, hidden_units)
        self.value_down = nn.Linear(input_shape, hidden_units)
        self.value_up = nn.Linear(hidden_units, input_shape)
        self.sqrt_d = torch.sqrt(torch.tensor(input_shape, dtype=torch.float32))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        K: torch.Tensor = self.keys(x)
        Q: torch.Tensor = self.queries(x)
        V: torch.Tensor = self.value_up(self.value_down(x))

        top = torch.matmul(Q, K.transpose(-2, -1))
        attention = top / self.sqrt_d
        attention = torch.nn.functional.softmax(attention, dim=-1)
        return torch.matmul(attention, V)

In [124]:
class MultiHeadAttention(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, num_heads: int) -> None:
        super().__init__()
        self.heads = nn.ModuleList([AttentionHead(input_shape, hidden_units) for _ in range(num_heads)])
        self.linear = nn.Linear(hidden_units * num_heads, input_shape)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.linear(torch.cat([head(x) for head in self.heads], dim=-1))

In [125]:
class EncoderLayer(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, num_heads: int) -> None:
        super().__init__()
        self.multi_head_attention = MultiHeadAttention(input_shape, hidden_units, num_heads)
        self.linear = nn.Linear(input_shape, input_shape)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        before = x.clone()
        x = self.multi_head_attention(x)
        x = torch.nn.functional.layer_norm(x + before, x.shape)
        before = x.clone()
        x = self.linear(x)
        x = torch.nn.functional.layer_norm(x + before, x.shape)
        return x

In [132]:
class CustomModel(nn.Module):
    def __init__(
        self, input_shape: int, embedding_dim: int, hidden_units: int, num_encoders: int, num_heads: int
    ) -> None:
        super().__init__()
        self.embedding = nn.Embedding(
            num_embeddings=input_shape, embedding_dim=embedding_dim
        )
        self.encoders = nn.ModuleList(
            [EncoderLayer(embedding_dim, hidden_units, num_heads) for _ in range(num_encoders)]
        )
        
        self.final_layer = nn.Linear(in_features=hidden_units*CONTEXT_WINDOW, out_features=input_shape)

    def forward(self, x):
        x = self.embedding(x)
        for encoder in self.encoders:
            x = encoder(x)
        x = x.view(x.shape[0], -1)
        x = self.final_layer(x)
        return x

In [145]:
vocab_size = len(all_chars)
model_0 = CustomModel(
    input_shape=vocab_size,
    embedding_dim=100,
    hidden_units=100,
    num_encoders=10,
    num_heads=10
).to(device)

In [146]:
total_params = 0
for x in model_0.parameters():
    total_params += np.array(x.shape).prod()
    
f"{total_params:_}" # 40 961 600

'5_414_727'

In [147]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter("../runs/transformer")

In [148]:
# make example prediction

model_0.eval()
with torch.inference_mode():
    for X, y in test_dataloader:
        y_preds = model_0(X)
        print(X.shape, y_preds.shape, y.shape)
        break

torch.Size([10, 100]) torch.Size([10, 27]) torch.Size([10])


In [149]:
def train_step(
    model: nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: nn.Module,
    optimizer: torch.optim.Optimizer,
    accuracy_fn,
):
    train_loss, train_acc = 0, 0
    for X, y in tqdm(dataloader):
        model.train()
        y_logits = model(X)
        y_preds = torch.argmax(torch.softmax(y_logits, dim=1), dim=1)
        loss = loss_fn(y_logits, y)
        acc = accuracy_fn(y_true=y.cpu(), y_pred=y_preds.cpu())

        train_loss += loss.item()
        train_acc += acc

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss /= len(dataloader)
    train_acc /= len(dataloader)

    return train_loss, train_acc

In [150]:
def test_step(
    model: nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: nn.Module,
    accuracy_fn,
):
    test_loss, test_acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for X, y in tqdm(dataloader):
            y_logits = model(X)
            y_preds = torch.argmax(torch.softmax(y_logits, dim=1), dim=1)
            loss = loss_fn(y_logits, y)
            acc = accuracy_fn(y_true=y.cpu(), y_pred=y_preds.cpu())

            test_loss += loss.item()
            test_acc += acc

    test_loss /= len(dataloader)
    test_acc /= len(dataloader)

    return test_loss, test_acc

In [151]:
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm


def train_model(
    model: nn.Module,
    train_dataloader: torch.utils.data.DataLoader,
    test_dataloader: torch.utils.data.DataLoader,
    loss_fn: nn.Module,
    optimizer: torch.optim.Optimizer,
    accuracy_score: callable,
    epochs: int = 3,
) -> dict[str, list[float]]:

    results = {
        "train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": [],
    }
    for epoch in tqdm(range(epochs)):

        train_loss, train_acc = train_step(
            model=model,
            dataloader=train_dataloader,
            loss_fn=loss_fn,
            optimizer=optimizer,
            accuracy_fn=accuracy_score,
        )

        test_loss, test_acc = test_step(
            model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn,
            accuracy_fn=accuracy_score,
        )

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    return results

In [152]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_0.parameters(), lr=0.01)

model_0_train_results = train_model(
    model=model_0,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    loss_fn=loss_fn,
    optimizer=optimizer,
    accuracy_score=accuracy_score,
    epochs=3,
)

KeyboardInterrupt: 

In [ ]:
writer = SummaryWriter("../runs/model_0")

for epoch in range(len(model_0_train_results["train_loss"])):
    writer.add_scalars(
        "loss",
        {
            "train_loss": model_0_train_results["train_loss"][epoch],
            "test_loss": model_0_train_results["test_loss"][epoch],
        },
        epoch,
    )
    writer.add_scalars(
        "accuracy",
        {
            "train_acc": model_0_train_results["train_acc"][epoch],
            "test_acc": model_0_train_results["test_acc"][epoch],
        },
        epoch,
    )

writer.close()

In [ ]:
plt.figure(figsize=(12, 6))

# Plotting loss and accuracy
for i, metric in enumerate(["loss", "acc"]):
    plt.subplot(1, 2, i + 1)
    for key in model_0_train_results:
        if metric in key:
            plt.plot(
                range(len(model_0_train_results[key])),
                model_0_train_results[key],
                label=key,
            )
    plt.xlabel("Epoch")
    plt.ylabel(metric.capitalize())
    plt.title(f"Train {metric.capitalize()}")
    plt.legend()

plt.tight_layout()
plt.show()